# Correlation analysis

In [3]:
import pickle
import scipy
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from scipy.stats import pearsonr
from scipy.stats import ttest_ind
from scipy.stats import ttest_rel

import warnings
warnings.filterwarnings("ignore")

import sys

## Load data

In [65]:
dataset = 'test'
test = False if dataset == 'train' else False

Loading training data

In [66]:
results_ern_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_{dataset}.pkl")
results_crn_lat_demo_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_{dataset}.pkl")

train_datasets = [results_ern_lat_demo_df, results_crn_lat_demo_df]

Loading testing data

In [67]:
results_ern_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/ern_cov_fal_models_test.pkl")
results_crn_lat_demo_test_df = pd.read_pickle(f"../data/models_pickles_new_dass/crn_cov_fal2_models_test.pkl")

# fill nan
results_crn_lat_demo_test_df['e_LT_F2_C'] = results_crn_lat_demo_test_df['e_LT_F2_C'].fillna(results_crn_lat_demo_test_df['e_LT_F2_C'].mean())

test_datasets = [results_ern_lat_demo_test_df, results_crn_lat_demo_test_df]

### Create delta ERN - CRN measure

In [68]:
datasets = test_datasets if test else train_datasets

ern_values = datasets[0]['e_ERN'].to_numpy()
crn_values = datasets[1]['e_CRN'].to_numpy()

ern_crn_diff_values = ern_values - crn_values
ern_crn_diff_df = pd.DataFrame(ern_crn_diff_values.reshape(-1,1), columns=['ERN\CRN delta'])

# Add new column to dataset
ern_delta_data_df = pd.concat([ern_crn_diff_df, datasets[0]], axis=1).drop(columns='e_ERN')

datasets.append(ern_delta_data_df)

Drop unnecessery columns

In [69]:
datasets = [dataset.drop(columns=['Sex', 'WASH', 'NEU'])for dataset in datasets]

## Create correlation tables

In [71]:
mapping = {
    "RRQ": "1Q",
    "DASS-21 Stress": "2Q",
    "DASS-21 Dep": "3Q",
    "STAI-T": "4Q",
    "STAI-S Diff": '5Q',
    "BIS": "6Q",
    "OBSESS": "7Q",
    "HOARD": "8Q",
    "ORD": "9Q",
    "CHECK": "10Q",
    "WBSI": "11Q",
    "IUS-P": "12Q",
    "IUS-I": "13Q",
    "SES": "14Q",
    'BAS_D': "15Q",
    'BAS_PRZY': "16Q",
    'BAS_NAG': "17Q",
    'INDEC_F': "18Q",
    'PUN': "19Q",
    'REW': "20Q",
    'HARM': "21Q",
    'T-CTR': "22Q",
    "OT": "23Q",
    'OB_PERF': "24Q",
    'PS': "25Q",
    'G_SE': "26Q",
    'AMB': "27Q",
    'PRED': "28Q",
    'STAND': "29Q",   
    "Age": "1C",
    "Handness": "2C",
    'e_ERN': "1E",
    'e_LT_F': "4C",
    'performance': "3C",
    'e_CRN': "2E",
    'e_LT_F2_C': "4C",
}

In [72]:
def corr_with_p(df):
    df = df.rename(columns=mapping)
    rho = df.corr()
    pval = df.corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
    p = pval.applymap(lambda x: ''.join(['*' for t in [0.01,0.05,0.1] if x<=t]))
    return rho.round(2).astype(str) + p, rho

In [73]:
corr_p_tables = []

for index, this_dataset in enumerate(datasets):
    print(index)
    corr_p, corr = corr_with_p(this_dataset)
    corr_p.to_csv(f'../data/correlation_matrixes/{dataset}/correlation_matrix_{index}.csv')
    display(corr_p)

0


,1E,4C,1C,2C,1Q,2Q,3Q,4Q,5Q,6Q,...,22Q,24Q,25Q,27Q,28Q,29Q,12Q,13Q,14Q,3C
1E,1.0***,-0.42***,0.05,-0.05,-0.06,0.1,0.22**,0.07,0.0,0.07,...,-0.07,-0.0,-0.05,0.13,0.09,-0.0,-0.02,-0.06,-0.05,-0.22**
4C,-0.42***,1.0***,0.11,-0.04,0.11,0.02,-0.07,-0.05,-0.05,-0.01,...,-0.03,-0.06,0.03,-0.02,0.02,-0.11,-0.03,0.09,-0.02,0.14
1C,0.05,0.11,1.0***,-0.01,0.06,0.01,0.02,-0.06,-0.0,-0.02,...,-0.14,0.04,0.1,0.06,0.16*,-0.03,0.11,0.07,0.0,0.1
2C,-0.05,-0.04,-0.01,1.0***,0.13,-0.15,-0.08,-0.02,0.15,-0.03,...,-0.05,0.03,-0.04,-0.09,-0.14,-0.11,-0.07,-0.12,0.1,0.13
1Q,-0.06,0.11,0.06,0.13,1.0***,0.4***,0.36***,0.59***,-0.05,0.55***,...,0.19**,0.35***,0.01,0.47***,0.28***,0.11,0.43***,0.55***,-0.51***,0.17*
2Q,0.1,0.02,0.01,-0.15,0.4***,1.0***,0.63***,0.7***,0.12,0.45***,...,0.19**,0.3***,0.09,0.18*,0.08,0.15,0.46***,0.47***,-0.48***,-0.02
3Q,0.22**,-0.07,0.02,-0.08,0.36***,0.63***,1.0***,0.7***,0.18*,0.39***,...,0.33***,0.27***,-0.08,0.21**,0.14,0.15,0.4***,0.54***,-0.67***,-0.03
4Q,0.07,-0.05,-0.06,-0.02,0.59***,0.7***,0.7***,1.0***,0.11,0.66***,...,0.33***,0.39***,-0.1,0.34***,0.16*,0.15,0.44***,0.66***,-0.74***,0.11
5Q,0.0,-0.05,-0.0,0.15,-0.05,0.12,0.18*,0.11,1.0***,0.08,...,-0.1,0.05,0.15,0.01,-0.11,0.08,0.08,0.02,-0.17*,-0.1
6Q,0.07,-0.01,-0.02,-0.03,0.55***,0.45***,0.39***,0.66***,0.08,1.0***,...,0.17*,0.31***,-0.11,0.53***,0.19**,0.0,0.39***,0.48***,-0.52***,0.0


1


,2E,1C,2C,1Q,2Q,3Q,4Q,5Q,6Q,15Q,...,22Q,24Q,25Q,27Q,28Q,29Q,12Q,13Q,14Q,3C
2E,1.0***,-0.02,0.01,0.04,0.12,0.21**,0.05,0.08,0.2**,0.04,...,-0.01,0.01,0.02,0.24**,0.22**,-0.04,0.03,-0.04,-0.03,-0.15
1C,-0.02,1.0***,-0.01,0.06,0.01,0.02,-0.06,-0.0,-0.02,0.02,...,-0.14,0.04,0.1,0.06,0.16*,-0.03,0.11,0.07,0.0,0.1
2C,0.01,-0.01,1.0***,0.13,-0.15,-0.08,-0.02,0.15,-0.03,-0.05,...,-0.05,0.03,-0.04,-0.09,-0.14,-0.11,-0.07,-0.12,0.1,0.13
1Q,0.04,0.06,0.13,1.0***,0.4***,0.36***,0.59***,-0.05,0.55***,-0.16*,...,0.19**,0.35***,0.01,0.47***,0.28***,0.11,0.43***,0.55***,-0.51***,0.17*
2Q,0.12,0.01,-0.15,0.4***,1.0***,0.63***,0.7***,0.12,0.45***,-0.03,...,0.19**,0.3***,0.09,0.18*,0.08,0.15,0.46***,0.47***,-0.48***,-0.02
3Q,0.21**,0.02,-0.08,0.36***,0.63***,1.0***,0.7***,0.18*,0.39***,-0.11,...,0.33***,0.27***,-0.08,0.21**,0.14,0.15,0.4***,0.54***,-0.67***,-0.03
4Q,0.05,-0.06,-0.02,0.59***,0.7***,0.7***,1.0***,0.11,0.66***,-0.2**,...,0.33***,0.39***,-0.1,0.34***,0.16*,0.15,0.44***,0.66***,-0.74***,0.11
5Q,0.08,-0.0,0.15,-0.05,0.12,0.18*,0.11,1.0***,0.08,-0.03,...,-0.1,0.05,0.15,0.01,-0.11,0.08,0.08,0.02,-0.17*,-0.1
6Q,0.2**,-0.02,-0.03,0.55***,0.45***,0.39***,0.66***,0.08,1.0***,-0.34***,...,0.17*,0.31***,-0.11,0.53***,0.19**,0.0,0.39***,0.48***,-0.52***,0.0
15Q,0.04,0.02,-0.05,-0.16*,-0.03,-0.11,-0.2**,-0.03,-0.34***,1.0***,...,0.05,0.0,0.39***,-0.19*,-0.11,0.29***,-0.05,-0.05,0.31***,-0.03


2


,ERN\CRN delta,4C,1C,2C,1Q,2Q,3Q,4Q,5Q,6Q,...,22Q,24Q,25Q,27Q,28Q,29Q,12Q,13Q,14Q,3C
ERN\CRN delta,1.0***,-0.18*,0.07,-0.07,-0.1,-0.0,0.06,0.04,-0.07,-0.09,...,-0.07,-0.02,-0.07,-0.06,-0.1,0.03,-0.05,-0.04,-0.03,-0.11
4C,-0.18*,1.0***,0.11,-0.04,0.11,0.02,-0.07,-0.05,-0.05,-0.01,...,-0.03,-0.06,0.03,-0.02,0.02,-0.11,-0.03,0.09,-0.02,0.14
1C,0.07,0.11,1.0***,-0.01,0.06,0.01,0.02,-0.06,-0.0,-0.02,...,-0.14,0.04,0.1,0.06,0.16*,-0.03,0.11,0.07,0.0,0.1
2C,-0.07,-0.04,-0.01,1.0***,0.13,-0.15,-0.08,-0.02,0.15,-0.03,...,-0.05,0.03,-0.04,-0.09,-0.14,-0.11,-0.07,-0.12,0.1,0.13
1Q,-0.1,0.11,0.06,0.13,1.0***,0.4***,0.36***,0.59***,-0.05,0.55***,...,0.19**,0.35***,0.01,0.47***,0.28***,0.11,0.43***,0.55***,-0.51***,0.17*
2Q,-0.0,0.02,0.01,-0.15,0.4***,1.0***,0.63***,0.7***,0.12,0.45***,...,0.19**,0.3***,0.09,0.18*,0.08,0.15,0.46***,0.47***,-0.48***,-0.02
3Q,0.06,-0.07,0.02,-0.08,0.36***,0.63***,1.0***,0.7***,0.18*,0.39***,...,0.33***,0.27***,-0.08,0.21**,0.14,0.15,0.4***,0.54***,-0.67***,-0.03
4Q,0.04,-0.05,-0.06,-0.02,0.59***,0.7***,0.7***,1.0***,0.11,0.66***,...,0.33***,0.39***,-0.1,0.34***,0.16*,0.15,0.44***,0.66***,-0.74***,0.11
5Q,-0.07,-0.05,-0.0,0.15,-0.05,0.12,0.18*,0.11,1.0***,0.08,...,-0.1,0.05,0.15,0.01,-0.11,0.08,0.08,0.02,-0.17*,-0.1
6Q,-0.09,-0.01,-0.02,-0.03,0.55***,0.45***,0.39***,0.66***,0.08,1.0***,...,0.17*,0.31***,-0.11,0.53***,0.19**,0.0,0.39***,0.48***,-0.52***,0.0
